# Test on Metabolic data

### Import the libraries

In [62]:
import math
import torch
import torch.nn as nn
import torchvision.models as models 
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import os
import ssl
import xml.etree.ElementTree as ET
import matplotlib
import plotly.express as px

from scipy.spatial import distance as dist 
from sklearn.manifold import TSNE 
from sklearn.decomposition import PCA
import umap.umap_ as umap
from PIL import Image
from mpl_toolkits.axes_grid1 import make_axes_locatable #typo "axesy" -> "axes"


In [2]:
from utils import dis_of_dis_transform

### Import the data for tNSE

In [8]:
file_dir = "/file_path/for/your/data/folder"

file_path = "/file_path/for/your/data.csv"

if os.path.isfile(file_path):
    print("File exists.")
else:
    print("File does not exist.")


File exists.


In [11]:
dataset = pd.read_excel('/path/file.xlsx', sheet_name='name')

In [13]:
dataset.shape

(10025, 24)

In [12]:
dataset.head()

,Mouse ID,1536-2,1531-2,1526-1,1536-4,1534-6,1536-2.1,1536-3,1528-1,1534-6.1,...,1534-3,1534-4,1524-2,1531-2.1,1536-3.1,1531-1.1,1534-1,1536-1,1524-1,1536-4.1
0,Group,A,B,C,C,A,C,A,D,C,...,C,C,D,D,C,D,C,C,D,A
1,Class,Control (FVB),DM1 (PBS),DM1 (PBS),Control (FVB),Control (FVB),Control (FVB),Control (FVB),DM1 (PBS),Control (FVB),...,Control (FVB),Control (FVB),DM1 (PBS),DM1 (PBS),Control (FVB),DM1 (PBS),Control (FVB),Control (FVB),DM1 (PBS),Control (FVB)
2,Muscle,Quadriceps,Quadriceps,Gastrocnemius,Gastrocnemius,Quadriceps,Gastrocnemius,Quadriceps,Gastrocnemius,Gastrocnemius,...,Gastrocnemius,Gastrocnemius,Gastrocnemius,Gastrocnemius,Gastrocnemius,Gastrocnemius,Gastrocnemius,Gastrocnemius,Gastrocnemius,Quadriceps
3,Weight,0.0412,0.0493,0.025,0.0301,0.0372,0.0261,0.0482,0.0345,0.0275,...,0.0279,0.0445,0.0341,0.0421,0.0322,0.0369,0.0416,0.0325,0.0423,0.0434
4,-0.499,-0.005875,0.001648,0.001007,-0.003127,-0.002427,-0.005537,0.001492,0.003237,0.002838,...,0.003032,0.001374,0.000402,-0.003044,0.001463,-0.005275,0.001961,-0.001512,-0.001725,-0.00255


#### Take care of missing values

In [14]:
np.where(dataset.isnull())

(array([2529, 2529, 2529, ..., 2915, 2915, 2915]),
 array([ 1,  2,  3, ..., 21, 22, 23]))

In [15]:
#remove rows with missing values
dataset = dataset.dropna()

In [16]:
np.where(dataset.isnull()) #doublecheck that there is no missing value now

(array([], dtype=int64), array([], dtype=int64))

In [17]:
dataset.shape

(9638, 24)

Transpose the table so that Mouse IDs are in rows, and the charecteristics of mouses and ppms are the features of each mice vector

In [21]:
df = dataset.set_index('Mouse ID').T.rename_axis('Mouse ID').rename_axis(None, axis=1).reset_index()
df.head()

,Mouse ID,Group,Class,Muscle,Weight,-0.499,-0.498,-0.497,-0.496,-0.495,...,Alanine Normalized,AMP-ADP Concentration,AMP-ADP Normalized to weight,AMP-ADP Normalized,Glycine Concentration,Glycine Normalized to weight,Glycine Normalized,Fumarate Concentration,Fumarate Normalized to weight,Fumarate Normalized
0,1536-2,A,Control (FVB),Quadriceps,0.0412,-0.005875,-0.005942,-0.003065,-0.002257,0.00011,...,0.47865,0.065522,1.59034,0.59378,0.014188,0.344363,0.452138,0.000737,0.017889,0.570496
1,1531-2,B,DM1 (PBS),Quadriceps,0.0493,0.001648,0.005415,-0.001195,0.000711,0.000623,...,1.232955,0.122405,2.482856,0.927016,0.025264,0.512459,0.672842,0.004125,0.083664,2.668103
2,1526-1,C,DM1 (PBS),Gastrocnemius,0.025,0.001007,-0.000439,-0.001736,-0.003978,-0.00409,...,0.382666,0.030953,1.238107,0.422312,0.006607,0.264274,0.310258,0.000376,0.015027,0.421261
3,1536-4,C,Control (FVB),Gastrocnemius,0.0301,-0.003127,0.001426,-0.005096,-0.000064,-0.002517,...,0.747679,0.063844,2.121048,0.723478,0.023231,0.771797,0.906091,0.000362,0.012038,0.337452
4,1534-6,A,Control (FVB),Quadriceps,0.0372,-0.002427,0.001221,-0.002969,-0.00186,-0.000444,...,0.746956,0.07924,2.130117,0.795315,0.019148,0.514725,0.675818,0.001297,0.034854,1.111524


Now we have 23 vectors in a 9639-dimentional space

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Columns: 9639 entries, Mouse ID to Fumarate Normalized
dtypes: object(9639)
memory usage: 1.7+ MB


The 'dtype' is type of values stored in our features, obect means text, and it's ok for the qualitative features like 'muscles', but quantitative features should be real numbers

In [23]:
#extract the columns with 'object' type values
columns_to_transform = list(df.select_dtypes('object'))
print(columns_to_transform)

['Mouse ID', 'Group', 'Class', 'Muscle', 'Weight', -0.499, -0.498, -0.497, -0.496, -0.495, -0.494, -0.493, -0.492, -0.491, -0.49, -0.489, -0.488, -0.487, -0.486, -0.485, -0.484, -0.483, -0.482, -0.481, -0.48, -0.479, -0.478, -0.477, -0.476, -0.475, -0.474, -0.473, -0.472, -0.471, -0.47, -0.469, -0.468, -0.467, -0.466, -0.465, -0.464, -0.463, -0.462, -0.461, -0.46, -0.459, -0.458, -0.457, -0.456, -0.455, -0.454, -0.453, -0.452, -0.451, -0.45, -0.449, -0.448, -0.447, -0.446, -0.445, -0.444, -0.443, -0.442, -0.441, -0.44, -0.439, -0.438, -0.437, -0.436, -0.435, -0.434, -0.433, -0.432, -0.431, -0.43, -0.429, -0.428, -0.427, -0.426, -0.425, -0.424, -0.423, -0.422, -0.421, -0.42, -0.419, -0.418, -0.417, -0.416, -0.415, -0.414, -0.413, -0.412, -0.411, -0.41, -0.409, -0.408, -0.407, -0.406, -0.405, -0.404, -0.403, -0.402, -0.401, -0.4, -0.399, -0.398, -0.397, -0.396, -0.395, -0.394, -0.393, -0.392, -0.391, -0.39, -0.389, -0.388, -0.387, -0.386, -0.385, -0.384, -0.383, -0.382, -0.381, -0.38, -0

In [24]:
#force all the features to numerilac values except for the first 4 features

for i in columns_to_transform[4:]:
    df[i] = pd.to_numeric(df[i], errors='coerce') #coerce -> invalid parsing will be set as NaN, other option is 'ignore' -> invalid parsing will return the input

Now we should have 4 text features and the rest should be float (=real numbers)

In [25]:
#double check the feature types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Columns: 9639 entries, Mouse ID to Fumarate Normalized
dtypes: float64(9635), object(4)
memory usage: 1.7+ MB


#### Save the file

In [32]:
#df.to_csv(file_dir + "NARMYD-DATA-cleared.csv", index=False)


#### Import a previosly cleared and preprocessed file

In [33]:
data_to_test = pd.read_csv(file_dir + "NARMYD-DATA-cleared-one-hot-encoded-labels-removed.csv")

In [34]:
data_to_test.head()

,Weight,-0.499,-0.498,-0.497,-0.496,-0.495,-0.494,-0.493,-0.492,-0.491,...,AMP-ADP Concentration,AMP-ADP Normalized to weight,AMP-ADP Normalized,Glycine Concentration,Glycine Normalized to weight,Glycine Normalized,Fumarate Concentration,Fumarate Normalized to weight,Fumarate Normalized,Muscle_Gastrocnemius
0,0.0412,-0.005875,-0.005942,-0.003065,-0.002257,0.000110,0.000063,0.000636,0.001314,0.001268,...,0.065522,1.590340,0.593780,0.014188,0.344363,0.452138,0.000737,0.017889,0.570496,False
1,0.0493,0.001648,0.005415,-0.001195,0.000711,0.000623,-0.001621,-0.000180,-0.002276,0.004298,...,0.122405,2.482856,0.927016,0.025264,0.512459,0.672842,0.004125,0.083664,2.668103,False
2,0.0250,0.001007,-0.000439,-0.001736,-0.003978,-0.004090,-0.004596,-0.001709,-0.001969,-0.002797,...,0.030953,1.238107,0.422312,0.006607,0.264274,0.310258,0.000376,0.015027,0.421261,True
3,0.0301,-0.003127,0.001426,-0.005096,-0.000064,-0.002517,-0.005557,-0.001458,-0.003545,0.004713,...,0.063844,2.121048,0.723478,0.023231,0.771797,0.906091,0.000362,0.012038,0.337452,True
4,0.0372,-0.002427,0.001221,-0.002969,-0.001860,-0.000444,-0.003403,0.004813,0.003745,0.007952,...,0.079240,2.130117,0.795315,0.019148,0.514725,0.675818,0.001297,0.034854,1.111524,False


In [57]:
data_to_visualize = pd.read_csv(file_dir + 'NARMYD-DATA-cleared-one-hot-encoded.csv')

In [58]:
data_to_visualize.head()

,Mouse ID,Group,Class,Muscle,Weight,-0.499,-0.498,-0.497,-0.496,-0.495,...,Alanine Normalized,AMP-ADP Concentration,AMP-ADP Normalized to weight,AMP-ADP Normalized,Glycine Concentration,Glycine Normalized to weight,Glycine Normalized,Fumarate Concentration,Fumarate Normalized to weight,Fumarate Normalized
0,1536-2,A,Control (FVB),Quadriceps,0.0412,-0.005875,-0.005942,-0.003065,-0.002257,0.000110,...,0.478650,0.065522,1.590340,0.593780,0.014188,0.344363,0.452138,0.000737,0.017889,0.570496
1,1531-2,B,DM1 (PBS),Quadriceps,0.0493,0.001648,0.005415,-0.001195,0.000711,0.000623,...,1.232955,0.122405,2.482856,0.927016,0.025264,0.512459,0.672842,0.004125,0.083664,2.668103
2,1526-1,C,DM1 (PBS),Gastrocnemius,0.0250,0.001007,-0.000439,-0.001736,-0.003978,-0.004090,...,0.382666,0.030953,1.238107,0.422312,0.006607,0.264274,0.310258,0.000376,0.015027,0.421261
3,1536-4,C,Control (FVB),Gastrocnemius,0.0301,-0.003127,0.001426,-0.005096,-0.000064,-0.002517,...,0.747679,0.063844,2.121048,0.723478,0.023231,0.771797,0.906091,0.000362,0.012038,0.337452
4,1534-6,A,Control (FVB),Quadriceps,0.0372,-0.002427,0.001221,-0.002969,-0.001860,-0.000444,...,0.746956,0.079240,2.130117,0.795315,0.019148,0.514725,0.675818,0.001297,0.034854,1.111524


### Distances calculation for pair of points

In [47]:
X = data_to_test.iloc[:, :-1].values

In [48]:
distances = dist.squareform(dist.pdist(X,'euclidean'))

In [49]:
distances.shape()

(23, 23)

In [75]:
# tsne/umap before dod transform
embedding_tsne = TSNE(n_components=3, metric='euclidean', init='pca', perplexity=15).fit_transform(X)
embedding_umap = umap.UMAP(metric='euclidean', n_neighbors=15, n_components=3).fit_transform(X)

In [76]:
data_to_visualize['tsne-3d-one'] = embedding_tsne[:,0]
data_to_visualize['tsne-3d-two'] = embedding_tsne[:,1]
data_to_visualize['tsne-3d-three'] = embedding_tsne[:,2]

In [77]:
data_to_visualize['UMAP-3d-one'] = embedding_umap[:,0]
data_to_visualize['UMAP-3d-two'] = embedding_umap[:,1]
data_to_visualize['UMAP-3d-three'] = embedding_umap[:,2]

In [78]:
fig_tsne_before_DoD = px.scatter_3d(data_to_visualize, x='tsne-3d-one', y='tsne-3d-two', z='tsne-3d-three', color='Group')

fig_tsne_before_DoD.show()

In [82]:
fig_UMAP_before_DoD = px.scatter_3d(data_to_visualize, x='UMAP-3d-one', y='UMAP-3d-two', z='UMAP-3d-three', color='Muscle')

fig_UMAP_before_DoD.show()

### Visualization

In [68]:
# tsne/umap on dod distance matrix 
ddistances = dis_of_dis_transform(distances, 5)
embedding_tsne_dod = TSNE(n_components=3, metric='precomputed', perplexity = 20, init='random').fit_transform(ddistances)
embedding_umap_dod = umap.UMAP(metric='precomputed', n_neighbors=10, n_components=3).fit_transform(ddistances)

/Users/vschastlivaia/miniforge3/envs/dimentionality-reduction/lib/python3.12/site-packages/umap/umap_.py:1857: UserWarning:

using precomputed metric; inverse_transform will be unavailable



In [69]:
data_to_visualize['tsne-3d-one-Dod'] = embedding_tsne_dod[:,0]
data_to_visualize['tsne-3d-two-Dod'] = embedding_tsne_dod[:,1]
data_to_visualize['tsne-3d-three-Dod'] = embedding_tsne_dod[:,2]

In [70]:
data_to_visualize['UMAP-3d-one-Dod'] = embedding_umap_dod[:,0]
data_to_visualize['UMAP-3d-two-Dod'] = embedding_umap_dod[:,1]
data_to_visualize['UMAP-3d-three-Dod'] = embedding_umap_dod[:,2]

In [71]:
fig_tsne_after_DoD = px.scatter_3d(data_to_visualize, x='tsne-3d-one-Dod', y='tsne-3d-two-Dod', z='tsne-3d-three-Dod', color='Class')

fig_tsne_after_DoD.show()

In [72]:
fig_UMAP_after_DoD = px.scatter_3d(data_to_visualize, x='UMAP-3d-one-Dod', y='UMAP-3d-two-Dod', z='UMAP-3d-three-Dod', color='Class')

fig_UMAP_after_DoD.show()

### Download the pretrained CNN

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context
alexnet = models.alexnet(pretrained=True)
vgg16 = models.vgg16(pretrained=True)

# define another class so that the network can output intermediate layer
class AlexNet(torch.nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(*list(alexnet.features.children()))
        self.avgpool = alexnet.avgpool
        self.classifier = nn.Sequential(*list(alexnet.classifier.children()))

    def forward(self, x):
        results = []
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        for ii, model in enumerate(self.classifier):
            x = model(x)
            if ii == 1: # extract fc6 response (classifier (1)) 
                results.append(x)
        return x, results         
    
# define another class so that the network can output intermediate layer
class VGG16(torch.nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(*list(vgg16.features.children()))
        self.avgpool = vgg16.avgpool
        self.classifier = nn.Sequential(*list(vgg16.classifier.children()))

    def forward(self, x):
        results = []
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        for ii, model in enumerate(self.classifier):
            x = model(x)
            if ii == 0: # extract fc6 response (classifier (1)) 
                results.append(x)
        return x, results      
    
    
# feedforward CNN model 
model=AlexNet()
# model=VGG16()
# model.eval()

In [ ]:
# path to the image folder
filenames  = os.listdir('/data/VOC2007/JPEGImages') # on GitHUB the folder is empty
input_batch = []
noise_batch = []
class_label = []
n_img = len(filenames)
for i in range(n_img):
    if filenames[i].endswith('.jpg'):
        img_id = filenames[i].strip('.jpg')
        # read the annotation files of images 
        tree = ET.parse('/data/VOC2007/Annotations/{}.xml'.format(img_id))  # on GitHUB the folder is empty
        root = tree.getroot()
        if len(root.findall('object'))==1:     # if only single object in the image 
            filename = '/data/VOC2007/JPEGImages/{}.jpg'.format(img_id)  # on GitHUB the folder is empty
            input_image = Image.open(filename)
            preprocess = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(227),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            input_tensor = preprocess(input_image)
            input_batch.append(input_tensor)
            noise_tensor = transforms.ToTensor()(transforms.functional.crop(input_image, 0, 0, 227, 227)) #top left corner
            noise_batch.append(noise_tensor)
            class_label.append(root[6][0].text)

input_batch = torch.stack(input_batch)
noise_batch = torch.stack(noise_batch)
total_batch = torch.cat([input_batch, noise_batch], dim=0)

# label_dict = {np.unique(class_label)[i]:i for i in range(len(np.unique(class_label)))}
n_img = len(class_label)
label_dict = {'aeroplane': 4,
 'bicycle': 2,
 'bird': 18,
 'boat': 5,
 'bottle': 10,
 'bus': 1,
 'car': 0,
 'cat': 13,
 'chair': 7,
 'cow': 15,
 'diningtable': 9,
 'dog': 14,
 'horse': 16,
 'motorbike': 3,
 'person': 19,
 'pottedplant': 11,
 'sheep': 17,
 'sofa': 8,
 'train': 6,
 'tvmonitor': 12}
fmt = matplotlib.ticker.FuncFormatter(lambda x, pos: list(label_dict.keys())[list(label_dict.values()).index(int(x))])
class_label_color = [label_dict[i] for i in class_label] 

In [ ]:
# image patches only 
# tsne on the intermediate layer response 
with torch.no_grad():
    output, intermediate_img = model(input_batch)

# Visualize the image classification by pretrained AlexNet 
output = torch.nn.functional.softmax(output, dim=1).detach().numpy() # softmax
# _ = [plt.plot(output[j,:]) for j in range(500)] # classification 500 examples    
intermediate_img = intermediate_img[0].detach().numpy()

# tsne/umap on intermediate layer response 
distances = dist.squareform(dist.pdist(intermediate_img,'euclidean'))
embedding_tsne = TSNE(n_components=2, metric='euclidean', init='pca', perplexity=20).fit_transform(intermediate_img)
embedding_umap = umap.UMAP(metric='euclidean', n_neighbors=100).fit_transform(intermediate_img)

In [ ]:
# visualization 
fig, axa = plt.subplots(1,3, figsize=(15,4))
distances_sorted = dist.squareform(dist.pdist(intermediate_img[np.argsort(class_label),:],'euclidean'))
q1, q3 = np.percentile(distances_sorted.flatten(),[25,75])
iqr = q3-q1
lower = q1 - (1.5*iqr)
upper = q3 + (1.5*iqr)
im1 = axa[0].imshow(distances_sorted, aspect='equal',cmap='hot'); 
axa[0].set_title('dissimilarity matrix')    
im1.set_clim(lower, upper)
plt.colorbar(im1,ax=axa[0], ticks=[0,math.floor(np.max(distances_sorted)*100)/100.0],cax=make_axes_locatable(axa[0]).append_axes("right", size="5%", pad=0.05))
axa[0].set_xticks([0, distances_sorted.shape[0]]); axa[0].set_yticks([0, distances_sorted.shape[0]])

im2 = axa[1].scatter(embedding_tsne[:n_img, 0], embedding_tsne[:n_img, 1],c=class_label_color,cmap='jet',marker='o',s = 5) 
plt.colorbar(im2, ax=axa[1], format=fmt,ticks=np.arange(20));
axa[1].set_xticklabels([]); axa[1].set_xticks([])
axa[1].set_yticklabels([]); axa[1].set_yticks([])
axa[1].set_title('tsne visualization')

im3 = axa[2].scatter(embedding_umap[:n_img, 0], embedding_umap[:n_img, 1],c=class_label_color,cmap='jet',marker='o',s = 5) 
plt.colorbar(im3, ax=axa[2], format=fmt,ticks=np.arange(20));
axa[2].set_xticklabels([]); axa[2].set_xticks([])
axa[2].set_yticklabels([]); axa[2].set_yticks([])
axa[2].set_title('umap visualization')

plt.tight_layout(); plt.show()

# dis of dis transform

In [ ]:
# DOD ONLY ON CLUSTER POINTS

k = 20 # size of neighborhood  
ddistances = dis_of_dis_transform(distances, n_neighbors = k)
embedding_tsne_dod = TSNE(n_components=2, metric='precomputed', perplexity=20).fit_transform(ddistances)
embedding_umap_dod = umap.UMAP(metric='precomputed', n_neighbors=100).fit_transform(ddistances)

In [ ]:
# Visualization 
fig, axa = plt.subplots(2,3, figsize=(15,8))

im1 = axa[0][0].imshow(distances, aspect='equal',cmap='hot'); 
axa[0][0].set_title('dissimilarity matrix')
plt.colorbar(im1,ax=axa[0][0],ticks=[0,math.floor(np.max(distances)*100)/100.0],cax=make_axes_locatable(axa[0][0]).append_axes("right", size="5%", pad=0.05))
axa[0][0].set_xticks([0, distances.shape[0]]); axa[0][0].set_yticks([0, distances.shape[0]])

im2 = axa[1][0].imshow(ddistances, aspect='equal',cmap='hot'); 
axa[1][0].set_title('dissimilarity matrix with dod')
plt.colorbar(im2,ax=axa[1][0],ticks=[0,math.floor(np.max(ddistances)*100)/100.0],cax=make_axes_locatable(axa[1][0]).append_axes("right", size="5%", pad=0.05))
axa[1][0].set_xticks([0, ddistances.shape[0]]); axa[1][0].set_yticks([0, ddistances.shape[0]])

im3 = axa[0][1].scatter(embedding_tsne[:n_img, 0], embedding_tsne[:n_img, 1],c=class_label_color,cmap='jet',marker='o',s = 5) 
plt.colorbar(im3, ax=axa[0][1], format=fmt,ticks=np.arange(20));
axa[0][1].set_title('tsne visualization');axa[0][1].legend(['nat img','noise'])
axa[0][1].set_xticklabels([]); axa[0][1].set_xticks([])
axa[0][1].set_yticklabels([]); axa[0][1].set_yticks([])

im4 = axa[1][1].scatter(embedding_tsne_dod[:n_img, 0], embedding_tsne_dod[:n_img, 1],c=class_label_color,cmap='jet',marker='o',s = 5) 
plt.colorbar(im4, ax=axa[1][1], format=fmt,ticks=np.arange(20));
axa[1][1].set_title('tsne visualization with dod');axa[1][1].legend(['nat img','noise'])
axa[1][1].set_xticklabels([]); axa[1][1].set_xticks([])
axa[1][1].set_yticklabels([]); axa[1][1].set_yticks([])

im5 = axa[0][2].scatter(embedding_umap[:n_img, 0], embedding_umap[:n_img, 1],c=class_label_color,cmap='jet',marker='o',s = 5) 
plt.colorbar(im5, ax=axa[0][2], format=fmt,ticks=np.arange(20));
axa[0][2].set_title('umap visualization');axa[0][2].legend(['nat img','noise'])
axa[0][2].set_xticklabels([]); axa[0][2].set_xticks([])
axa[0][2].set_yticklabels([]); axa[0][2].set_yticks([])

im6 = axa[1][2].scatter(embedding_umap_dod[:n_img, 0], embedding_umap_dod[:n_img, 1],c=class_label_color,cmap='jet',marker='o',s = 5) 
plt.colorbar(im6, ax=axa[1][2], format=fmt,ticks=np.arange(20));
axa[1][2].set_title('umap visualization with dod');axa[1][2].legend(['nat img','noise'])
axa[1][2].set_xticklabels([]); axa[1][2].set_xticks([])
axa[1][2].set_yticklabels([]); axa[1][2].set_yticks([])

plt.tight_layout(); plt.show()

In the absence of noise points, the application of distance-of-distance transformation introduces some distortion to the original manifold 

In [ ]:
# image patches with noise patches 
# tsne on the intermediate layer response 
with torch.no_grad():
    output, intermediate = model(total_batch)

# tsne/umap on intermediate layer response 
intermediate = intermediate[0].detach().numpy()
embedding_tsne = TSNE(n_components=2, metric='euclidean', init='pca',perplexity = 20).fit_transform(intermediate)
embedding_umap = umap.UMAP(metric='euclidean', n_neighbors=100).fit_transform(intermediate)

In [ ]:
# tsne/umap on dod distance matrix 
distances = dist.squareform(dist.pdist(intermediate,'euclidean'))
ddistances = dis_of_dis_transform(distances, 5)
embedding_tsne_dod = TSNE(n_components=2, metric='precomputed', perplexity = 20).fit_transform(ddistances)
embedding_umap_dod = umap.UMAP(metric='precomputed', n_neighbors=100).fit_transform(ddistances)

In [ ]:
#%% visualization
fig, axa = plt.subplots(2,3, figsize=(15,8))

im1 = axa[0][0].imshow(distances, aspect='equal',cmap='hot'); 
axa[0][0].set_title('dissimilarity matrix')
plt.colorbar(im1,ax=axa[0][0],ticks=[0,math.floor(np.max(distances)*100)/100.0],cax=make_axes_locatable(axa[0][0]).append_axes("right", size="5%", pad=0.05))
axa[0][0].set_xticks([0, distances.shape[0]]); axa[0][0].set_yticks([0, distances.shape[0]])

im2 = axa[1][0].imshow(ddistances, aspect='equal',cmap='hot'); 
axa[1][0].set_title('dissimilarity matrix with dod')
plt.colorbar(im2,ax=axa[1][0],ticks=[0,math.floor(np.max(ddistances)*100)/100.0],cax=make_axes_locatable(axa[1][0]).append_axes("right", size="5%", pad=0.05))
axa[1][0].set_xticks([0, ddistances.shape[0]]); axa[1][0].set_yticks([0, ddistances.shape[0]])

im3 = axa[0][1].scatter(embedding_tsne[:n_img, 0], embedding_tsne[:n_img, 1],c=class_label_color,cmap='jet',marker='o',s = 5) 
plt.colorbar(im3, ax=axa[0][1], format=fmt,ticks=np.arange(20));
axa[0][1].scatter(embedding_tsne[n_img:, 0], embedding_tsne[n_img:, 1],c='grey',marker='o',s = 5) 
axa[0][1].set_title('tsne visualization');axa[0][1].legend(['nat img','noise'])
axa[0][1].set_xticklabels([]); axa[0][1].set_xticks([])
axa[0][1].set_yticklabels([]); axa[0][1].set_yticks([])

im4 = axa[1][1].scatter(embedding_tsne_dod[:n_img, 0], embedding_tsne_dod[:n_img, 1],c=class_label_color,cmap='jet',marker='o',s = 5) 
plt.colorbar(im4, ax=axa[1][1], format=fmt,ticks=np.arange(20));
axa[1][1].scatter(embedding_tsne_dod[n_img:, 0], embedding_tsne_dod[n_img:, 1],c='grey',marker='o',s = 5) 
axa[1][1].set_title('tsne visualization with dod');axa[1][1].legend(['nat img','noise'])
axa[1][1].set_xticklabels([]); axa[1][1].set_xticks([])
axa[1][1].set_yticklabels([]); axa[1][1].set_yticks([])

im5 = axa[0][2].scatter(embedding_umap[:n_img, 0], embedding_umap[:n_img, 1],c=class_label_color,cmap='jet',marker='o',s = 5) 
plt.colorbar(im5, ax=axa[0][2], format=fmt,ticks=np.arange(20));
axa[0][2].scatter(embedding_umap[n_img:, 0], embedding_umap[n_img:, 1],c='grey',marker='o',s = 5) 
axa[0][2].set_title('umap visualization');axa[0][2].legend(['nat img','noise'])
axa[0][2].set_xticklabels([]); axa[0][2].set_xticks([])
axa[0][2].set_yticklabels([]); axa[0][2].set_yticks([])

im6 = axa[1][2].scatter(embedding_umap_dod[:n_img, 0], embedding_umap_dod[:n_img, 1],c=class_label_color,cmap='jet',marker='o',s = 5) 
plt.colorbar(im6, ax=axa[1][2], format=fmt,ticks=np.arange(20));
axa[1][2].scatter(embedding_umap_dod[n_img:, 0], embedding_umap_dod[n_img:, 1],c='grey',marker='o',s = 5) 
axa[1][2].set_title('umap visualization with dod');axa[1][2].legend(['nat img','noise'])
axa[1][2].set_xticklabels([]); axa[1][2].set_xticks([])
axa[1][2].set_yticklabels([]); axa[1][2].set_yticks([])

plt.tight_layout(); plt.show()

With distance-of-distance transformation, the noise cloud that was masking the clustering is now separated from the object manifold. 

In [ ]:
#%% REVIEW: DOD COMPARED WITH PCA PREP  

# pca preprocessing 
pca = PCA(n_components=min(intermediate.shape))
intermediate_pca = pca.fit_transform(intermediate)
# plt.plot(pca.explained_variance_ratio_)
n_pc = 5
intermediate_pca = intermediate_pca[:,:5]
distances_PCA = dist.squareform(dist.pdist(intermediate_pca[:,:5],'euclidean'))
embedding_tsne_PCA = TSNE(n_components=2, metric='euclidean', init='pca', perplexity= 20).fit_transform(intermediate_pca)
embedding_umap_PCA = umap.UMAP(n_components=2, metric='euclidean', n_neighbors=100).fit_transform(intermediate_pca)

In [ ]:
# visualization 
fig, axa = plt.subplots(2,3, figsize=(15,8))
im1 = axa[0][0].imshow(distances, aspect='equal',cmap='hot'); 
axa[0][0].set_title('dissimilarity matrix')
plt.colorbar(im1,ax=axa[0][0],ticks=[0,math.floor(np.max(distances)*100)/100.0],cax=make_axes_locatable(axa[0][0]).append_axes("right", size="5%", pad=0.05))
axa[0][0].set_xticks([0, distances.shape[0]]); axa[0][0].set_yticks([0, distances.shape[0]])

im2 = axa[1][0].imshow(distances_PCA, aspect='equal',cmap='hot'); 
axa[1][0].set_title('dissimilarity matrix with pca')
plt.colorbar(im2,ax=axa[1][0],ticks=[0,math.floor(np.max(distances_PCA)*100)/100.0],cax=make_axes_locatable(axa[1][0]).append_axes("right", size="5%", pad=0.05))
axa[1][0].set_xticks([0, distances_PCA.shape[0]]); axa[1][0].set_yticks([0, distances_PCA.shape[0]])

im3 = axa[0][1].scatter(embedding_tsne[:n_img, 0], embedding_tsne[:n_img, 1],c=class_label_color,cmap='jet',marker='o',s = 5) 
plt.colorbar(im3, ax=axa[0][1], format=fmt,ticks=np.arange(20));
axa[0][1].scatter(embedding_tsne[n_img:, 0], embedding_tsne[n_img:, 1],c='grey',marker='o',s = 5) 
axa[0][1].set_title('tsne visualization');axa[0][1].legend(['nat img','noise'])
axa[0][1].set_xticklabels([]); axa[0][1].set_xticks([])
axa[0][1].set_yticklabels([]); axa[0][1].set_yticks([])

im4 = axa[1][1].scatter(embedding_tsne_PCA[:n_img, 0], embedding_tsne_PCA[:n_img, 1],c=class_label_color,cmap='jet',marker='o',s = 5) 
plt.colorbar(im4, ax=axa[1][1], format=fmt,ticks=np.arange(20));
axa[1][1].scatter(embedding_tsne_PCA[n_img:, 0], embedding_tsne_PCA[n_img:, 1],c='grey',marker='o',s = 5) 
axa[1][1].set_title('tsne visualization with pca');axa[1][1].legend(['nat img','noise'])
axa[1][1].set_xticklabels([]); axa[1][1].set_xticks([])
axa[1][1].set_yticklabels([]); axa[1][1].set_yticks([])

im5 = axa[0][2].scatter(embedding_umap[:n_img, 0], embedding_umap[:n_img, 1],c=class_label_color,cmap='jet',marker='o',s = 5) 
plt.colorbar(im5, ax=axa[0][2], format=fmt,ticks=np.arange(20));
axa[0][2].scatter(embedding_umap[n_img:, 0], embedding_umap[n_img:, 1],c='grey',marker='o',s = 5) 
axa[0][2].set_title('umap visualization');axa[0][2].legend(['nat img','noise'])
axa[0][2].set_xticklabels([]); axa[0][2].set_xticks([])
axa[0][2].set_yticklabels([]); axa[0][2].set_yticks([])

im6 = axa[1][2].scatter(embedding_umap_PCA[:n_img, 0], embedding_umap_PCA[:n_img, 1],c=class_label_color,cmap='jet',marker='o',s = 5) 
plt.colorbar(im6, ax=axa[1][2], format=fmt,ticks=np.arange(20));
axa[1][2].scatter(embedding_umap_PCA[n_img:, 0], embedding_umap_PCA[n_img:, 1],c='grey',marker='o',s = 5) 
axa[1][2].set_title('umap visualization with pca');axa[1][2].legend(['nat img','noise'])
axa[1][2].set_xticklabels([]); axa[1][2].set_xticks([])
axa[1][2].set_yticklabels([]); axa[1][2].set_yticks([])

plt.tight_layout(); plt.show()

Other commonly used techniques including PCA preprocessing and PCA initialization cannot achieve the separation of noise points as well as distance-of-distance transformation.